In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import largestinteriorrectangle as lir

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
def find_max_rectangle_lir(binary_mask, contour = None):
    """
    Find maximum rectangle using LiR library.
    
    Args:
        binary_mask: Binary mask (numpy array of 0s and 1s)
    Returns:
        tuple: (x, y, width, height) of maximum rectangle
    """
    # Find maximum interior rectangle using LiR
    rect = lir.lir(binary_mask, contour)
    return rect

def visualize_max_rectangle(image, rect, color=(0, 255, 0), thickness=10):
    """
    Draw rectangle on image for visualization.
    
    Args:
        image: Input image
        rect: Rectangle tuple (x, y, w, h)
        color: BGR color tuple
        thickness: Line thickness
    Returns:
        Image with rectangle drawn
    """
    result = image.copy()
    
    cv2.rectangle(result, lir.pt1(rect), lir.pt2(rect), color, thickness)
    return result

In [ ]:
def find_panorama_lir(panorama):
    # Convert to grayscale if needed
    if len(panorama.shape) == 3:
        gray = cv2.cvtColor(panorama, cv2.COLOR_BGR2GRAY)
    else:
        gray = panorama.copy()
    
    # Create mask for content detection
    mask = np.zeros(gray.shape, dtype=np.uint8)

    # Analyze image in blocks
    block_size = 16
    for y in range(0, gray.shape[0], block_size):
        for x in range(0, gray.shape[1], block_size):
            # Define block region
            block_height = min(block_size, gray.shape[0] - y)
            block_width = min(block_size, gray.shape[1] - x)
            roi = gray[y:y+block_height, x:x+block_width]

            # Calculate local statistics
            block_mean, block_std = cv2.meanStdDev(roi)
            
            # Check if block contains content
            if (block_std[0] == 0):
                mask[y:y+block_height, x:x+block_width] = 255

    # Clean up mask with morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    mask = cv2.bitwise_not(mask)
    mask_lir = mask > 0.5

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contour = contours[0][:, 0, :]
    
    largest_rec = find_max_rectangle_lir(mask_lir, contour)

    return largest_rec

In [ ]:
def crop_panorama(panorama, largest_rect, padding=0, content_threshold=0.05):
    """
    Automatically crop panorama to content area with padding.

    Args:
        panorama: Input image
        padding: Number of pixels to pad around content
        content_threshold: Threshold for content detection

    Returns:
        Cropped panorama image
    """
    
    x, y, w, h = largest_rect    
    mask = np.zeros(panorama.shape, np.uint8)
    mask[y:y+h,x:x+w] = panorama[y:y+h,x:x+w]
    return panorama[y:y+h, x:x+w]

In [ ]:
def process_panorama(panorama):
    largest_rec = find_panorama_lir(
        panorama,
    )
    final = crop_panorama(panorama, largest_rec)
    return final

In [ ]:
def correct_distortion(image, k1=, k2=0):
    """
    Correct lens distortion using radial distortion parameters.
    k1, k2: radial distortion coefficients
    """
    height, width = image.shape[:2]
    camera_matrix = np.array([[width, 0, width/2],
                            [0, width, height/2],
                            [0, 0, 1]], dtype=np.float32)
    dist_coeffs = np.array([k1, k2, 0, 0], dtype=np.float32)
    
    # Get optimal new camera matrix
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(
        camera_matrix, dist_coeffs, (width, height), 1, (width, height))
    
    # Undistort the image
    dst = cv2.undistort(image, camera_matrix, dist_coeffs, None, newcameramtx)
    
    return dst

In [ ]:
image_path = os.path.join(DATA_FOLDER, "distortion.png")
panorama = cv2.imread(image_path)
result = process_panorama(panorama)

In [ ]:
plt.imshow(panorama)

In [ ]:
plt.figure(figsize = (20,20))
plt.imshow(result)
print(result.shape)

In [ ]:
undistorted_pano = correct_distortion(panorama)
plt.imshow(undistorted_pano)
undist_result = process_panorama(undistorted_pano)

In [ ]:
plt.figure(figsize = (20,20))
plt.imshow(undist_result)
print(result.shape)

In [ ]:
for i in np.arange(-1, 1, 0.2):
    for j in np.arange(-1, 1, 0.2):
        undistorted_pano = correct_distortion(panorama, k1=i, k2=j)
        plt.imshow(undistorted_pano)
        plt.show()